### Reference: https://realpython.com/courses/interactive-data-visualization-python-bokeh/

In [ ]:
import numpy as np
import pandas as pd

from bokeh.io import output_file, output_notebook
from bokeh.layouts import column, row, gridplot
from bokeh.models import ColumnDataSource, CategoricalColorMapper, CDSView, GroupFilter, HoverTool, NumeralTickFormatter
from bokeh.models.widgets import Tabs, Panel
from bokeh.plotting import figure, show

### Generate a figure, store in empty static HTML file, and render in browser

In [ ]:
output_file('output_test_file.html', title='Empty Bokeh Figure')
fig = figure()
show(fig)

### Generate a figure and render inline in Jupyter Notebook

In [ ]:
output_notebook()
fig = figure()
show(fig)

### Prepare figure for data

In [ ]:
output_notebook()

fig = figure(
    background_fill_color='gray',
    background_fill_alpha=0.5,
    border_fill_color='blue',
    border_fill_alpha=0.25,
    plot_height=300,
    plot_width=500,
    x_axis_label='X Label',
    x_axis_type='datetime',
    x_axis_location='above',
    x_range=('2018-01-01', '2018-06-30'),
    y_axis_label='Y Label',
    y_axis_type='linear',
    y_axis_location='left',
    y_range=(0, 100),
    title='Example Figure',
    title_location='right',
    toolbar_location='below',
    tools='save',
)

fig.grid.grid_line_color = None

show(fig)

### Drawing data with glyphs

In [ ]:
x_data = [1, 2, 1]
y_data = [1, 1, 2]

In [ ]:
output_notebook()

fig = figure(
    title='Coordinates',
    plot_height=300,
    plot_width=300,
    x_range=(0,3),
    y_range=(0,3),
    toolbar_location=None,
)

fig.circle(x=x_data, y=y_data, color='green', size=10, alpha=0.5)

show(fig)

In [ ]:
day_num = np.linspace(1, 10, num=10)
daily_words = [450, 628, 488, 210, 287, 791, 508, 639, 397, 943]
cumulative_words = np.cumsum(daily_words)
cumulative_words

In [ ]:
output_notebook()

fig = figure(
    title='Cumulative Words',
    plot_height=400,
    plot_width=700,
    x_axis_label='Day Number',
    x_minor_ticks=2,
    y_axis_label='Words Written',
    y_range=(0, max(cumulative_words)+1000),
    toolbar_location=None,
)

fig.vbar(x=day_num, bottom=0, top=daily_words, color='blue', width=0.75, legend_label='Daily')
fig.line(x=day_num, y=cumulative_words, color='orange', line_width=1, legend_label='Cumulative')
fig.legend.location = 'top_left'

show(fig)

### Reading actual data
https://github.com/realpython/materials/tree/master/intro-to-bokeh/data/

In [ ]:
player_stats = pd.read_csv(
    '../data/2017-18_playerBoxScore.csv', 
    parse_dates=['gmDate'],
)
print(player_stats.shape)
player_stats.head(5)

In [ ]:
team_stats = pd.read_csv(
    '../data/2017-18_teamBoxScore.csv', 
    parse_dates=['gmDate'],
)
print(team_stats.shape)
team_stats.head(5)

In [ ]:
standings = pd.read_csv(
    '../data/2017-18_standings.csv', 
    parse_dates=['stDate'],
)
print(standings.shape)
standings.head(5)

In [ ]:
west_top_2 = (
    standings[ (standings['teamAbbr'] == 'HOU') | (standings['teamAbbr'] == 'GS') ]
        .loc[:, ['stDate', 'teamAbbr', 'gameWon']]
        .sort_values(['teamAbbr', 'stDate'])
) 

In [ ]:
west_top_2.head(5)

In [ ]:
west_top_2.tail(5)

In [ ]:
player_stats = pd.read_csv(
    '../data/2017-18_playerBoxScore.csv', 
    parse_dates=['gmDate'],
)

# Find all players who took at least 1 3pt shot
three_takers = player_stats[player_stats['play3PA'] > 0]

# Clean up names by placing them into one column
three_takers['name'] = [f'{p["playFNm"]} {p["playLNm"]}' for _, p in three_takers.iterrows()]

# Aggregate the total 3pt shots (A=attempts, M=made) for each player
three_takers = (three_takers
        .groupby('name')
        .sum()
        .loc[:, ['play3PA', 'play3PM']]
        .sort_values('play3PA', ascending=False)
)

# Filter out anyone who did not take at least 100 3pt shots
three_takers = three_takers[three_takers['play3PA'] >= 100].reset_index()

# Add a column with a calculated 3pt percentage
three_takers['pct3PM'] = three_takers['play3PM'] / three_takers['play3PA']

three_takers.sample(5)

In [ ]:
phi76ers_gm_stats = (
    team_stats[ (team_stats['teamAbbr'] == 'PHI') & (team_stats['seasTyp'] == 'Regular') ]
        .loc[:, ['gmDate', 'teamPTS', 'teamTRB', 'teamAST', 'teamTO', 'opptPTS']]
        .sort_values('gmDate')
)
phi76ers_gm_stats['game_num'] = range(1, len(phi76ers_gm_stats) + 1)

win_loss = []
for _, row in phi76ers_gm_stats.iterrows():
    if row['teamPTS'] > row['opptPTS']:
        win_loss.append('W')
    else:
        win_loss.append('L')
phi76ers_gm_stats['winLoss'] = win_loss

phi76ers_gm_stats.head(5)

In [ ]:
phi76ers_gm_stats_points = (
    team_stats[ (team_stats['teamAbbr'] == 'PHI') & (team_stats['seasTyp'] == 'Regular') ]
        .loc[:, ['gmDate', 'team2P%', 'team3P%', 'teamPTS', 'opptPTS']]
        .sort_values('gmDate')
)

phi76ers_gm_stats_points['game_num'] = range(1, len(phi76ers_gm_stats_points) + 1)

win_loss = []
for _, row in phi76ers_gm_stats_points.iterrows():
    if row['teamPTS'] > row['opptPTS']:
        win_loss.append('W')
    else:
        win_loss.append('L')
phi76ers_gm_stats_points['winLoss'] = win_loss

phi76ers_gm_stats_points.head(5)

### Using a ColumnDataSource object

From **dict**:  
```
data = {'growth': [1.5, 3.0, 7.6, 8.9], 'months': [10, 30, 60, 100]}
source = ColumnDataSource(data)
```

From **Pandas DataFrame**:  
```
data = pd.read_csv('data.csv')
source = ColumnDataSource(data)
```

From **Pandas groupby**:  
```
data = pd.read_csv('data.csv').groupby('name').sum().loc[:, ['points']]
source = ColumnDataSource(data)
```

In [ ]:
rockets_data = west_top_2[west_top_2['teamAbbr'] == 'HOU']
warriors_data = west_top_2[west_top_2['teamAbbr'] == 'GS']

rockets_cds = ColumnDataSource(rockets_data)
warriors_cds = ColumnDataSource(warriors_data)

In [ ]:
output_notebook()

fig = figure(
    title='Western Conference Top 2 Teams Wins Race, 2017-2018',
    plot_height=300,
    plot_width=600,
    x_axis_type='datetime',
    x_axis_label='Date',
    y_axis_label='Wins',
    toolbar_location=None
)

fig.step(x='stDate', y='gameWon', color='#CE1141', legend_label='Rockets', source=rockets_cds)
fig.step(x='stDate', y='gameWon', color='#006BB6', legend_label='Warriors', source=warriors_cds)
fig.legend.location = 'top_left'

show(fig)

### Using a GroupFilter and a CDSView

3 built-in filters to create views:
- Group Filter (select rows from a ColumnDataSource based on a categorical reference value)
- Index Filter (filter the ColumnDataSource via a list of integer indices)
- Boolean Filter (allows a list of boolean values, with True rows being selected)

In [ ]:
west_cds = ColumnDataSource(west_top_2)

# Use the same ColumnDataSource

# ... but have a CDSView for a specific filter
rockets_view_filter = GroupFilter(column_name='teamAbbr', group='HOU')
rockets_view = CDSView(source=west_cds, filters=[rockets_view_filter])

# ... but have a CDSView for a specific filter
warriors_view_filter = GroupFilter(column_name='teamAbbr', group='GS')
warriors_view = CDSView(source=west_cds, filters=[warriors_view_filter])

In [ ]:
output_notebook()

fig = figure(
    title='Western Conference Top 2 Teams Wins Race, 2017-2018',
    plot_height=300,
    plot_width=600,
    x_axis_type='datetime',
    x_axis_label='Date',
    y_axis_label='Wins',
    toolbar_location=None
)

fig.step(
    x='stDate', y='gameWon',
    source=west_cds,    # Same CDS
    view=rockets_view,  # Specific view
    color='#CE1141', 
    legend_label='Rockets')
fig.step(
    x='stDate', y='gameWon',
    source=west_cds,    # Same CDS
    view=warriors_view, # Specific view
    color='#006BB6', 
    legend_label='Warriors')
fig.legend.location = 'top_left'

show(fig)

### Using Column Layout for multiple visualizations

In [ ]:
# Instead of creating separate DF from standings for west and east top 2, use standings DF directly.

standings = pd.read_csv(
    '../data/2017-18_standings.csv', 
    parse_dates=['stDate'],
)
standings_cds = ColumnDataSource(standings)

rockets_view_filter = GroupFilter(column_name='teamAbbr', group='HOU')
rockets_view = CDSView(source=standings_cds, filters=[rockets_view_filter])

warriors_view_filter = GroupFilter(column_name='teamAbbr', group='GS')
warriors_view = CDSView(source=standings_cds, filters=[warriors_view_filter])

celtics_view_filter = GroupFilter(column_name='teamAbbr', group='BOS')
celtics_view = CDSView(source=standings_cds, filters=[celtics_view_filter])

raptors_view_filter = GroupFilter(column_name='teamAbbr', group='TOR')
raptors_view = CDSView(source=standings_cds, filters=[raptors_view_filter])

In [ ]:
west_fig = figure(
    title='Western Conference Top 2 Teams Wins Race, 2017-2018',
    plot_height=300,
    x_axis_type='datetime',
    x_axis_label='Date',
    y_axis_label='Wins',
    toolbar_location=None
)
west_fig.step(
    x='stDate', y='gameWon',
    source=standings_cds,
    view=rockets_view,
    color='#CE1141', 
    legend_label='Rockets')
west_fig.step(
    x='stDate', y='gameWon',
    source=standings_cds,
    view=warriors_view,
    color='#006BB6', 
    legend_label='Warriors')
west_fig.legend.location = 'top_left'

east_fig = figure(
    title='Eastern Conference Top 2 Teams Wins Race, 2017-2018',
    plot_height=300,
    x_axis_type='datetime',
    x_axis_label='Date',
    y_axis_label='Wins',
    toolbar_location=None
)
east_fig.step(
    x='stDate', y='gameWon',
    source=standings_cds,
    view=celtics_view,
    color='#007A33', 
    legend_label='Celtics')
east_fig.step(
    x='stDate', y='gameWon',
    source=standings_cds,
    view=raptors_view,
    color='#CE1141', 
    legend_label='Raptors')
east_fig.legend.location = 'top_left'

In [ ]:
output_file('output-west-east-col-layout.html')

show(column(west_fig, east_fig))

In [ ]:
output_file('output-west-east-row-layout.html')

show(row(west_fig, east_fig))

### Using Grid Layout for multiple visualizations

In [ ]:
# Instead of creating separate DF from standings for west and east top 2, use standings DF directly.

standings = pd.read_csv(
    '../data/2017-18_standings.csv', 
    parse_dates=['stDate'],
)
standings_cds = ColumnDataSource(standings)

rockets_view_filter = GroupFilter(column_name='teamAbbr', group='HOU')
rockets_view = CDSView(source=standings_cds, filters=[rockets_view_filter])

warriors_view_filter = GroupFilter(column_name='teamAbbr', group='GS')
warriors_view = CDSView(source=standings_cds, filters=[warriors_view_filter])

celtics_view_filter = GroupFilter(column_name='teamAbbr', group='BOS')
celtics_view = CDSView(source=standings_cds, filters=[celtics_view_filter])

raptors_view_filter = GroupFilter(column_name='teamAbbr', group='TOR')
raptors_view = CDSView(source=standings_cds, filters=[raptors_view_filter])

In [ ]:
west_fig = figure(
    title='Western Conference Top 2 Teams Wins Race, 2017-2018',
    plot_height=300,
    x_axis_type='datetime',
    x_axis_label='Date',
    y_axis_label='Wins',
    toolbar_location=None
)
west_fig.step(
    x='stDate', y='gameWon',
    source=standings_cds,
    view=rockets_view,
    color='#CE1141', 
    legend_label='Rockets')
west_fig.step(
    x='stDate', y='gameWon',
    source=standings_cds,
    view=warriors_view,
    color='#006BB6', 
    legend_label='Warriors')
west_fig.legend.location = 'top_left'

east_fig = figure(
    title='Eastern Conference Top 2 Teams Wins Race, 2017-2018',
    plot_height=300,
    x_axis_type='datetime',
    x_axis_label='Date',
    y_axis_label='Wins',
    toolbar_location=None
)
east_fig.step(
    x='stDate', y='gameWon',
    source=standings_cds,
    view=celtics_view,
    color='#007A33', 
    legend_label='Celtics')
east_fig.step(
    x='stDate', y='gameWon',
    source=standings_cds,
    view=raptors_view,
    color='#CE1141', 
    legend_label='Raptors')
east_fig.legend.location = 'top_left'

In [ ]:
# Reduce the width for both figures
east_fig.plot_width = west_fig.plot_width = 300

# Add title for both figures
east_fig.title.text = 'Eastern Conference'
west_fig.title.text = 'Western Conference'

In [ ]:
output_file('output-west-east-grid-layout-1-row.html')

east_and_west_gridplot = gridplot(
    [[east_fig, west_fig]],
    toolbar_location='right',  # Can now have a common toolbar for *all* the figures
)

show(east_and_west_gridplot)

In [ ]:
output_file('output-west-east-grid-layout-2-rows.html')

east_and_west_gridplot = gridplot(
    [
        [east_fig, None],
        [None, west_fig],
    ],
    toolbar_location='right',  # Can now have a common toolbar for *all* the figures
)

show(east_and_west_gridplot)

### Using Tabbed Layout for multiple visualizations

```
from bokeh.models.widgets import Tabs, Panel

panel_1 = Panel(child=example_fig_1, title='Example #1')
panel_2 = Panel(child=example_fig_2, title='Example #2')

my_tabs = Tabs(tabs=[panel_1, panel_2])

show(my_tabs)
```

In [ ]:
# Instead of creating separate DF from standings for west and east top 2, use standings DF directly.

standings = pd.read_csv(
    '../data/2017-18_standings.csv', 
    parse_dates=['stDate'],
)
standings_cds = ColumnDataSource(standings)

rockets_view_filter = GroupFilter(column_name='teamAbbr', group='HOU')
rockets_view = CDSView(source=standings_cds, filters=[rockets_view_filter])

warriors_view_filter = GroupFilter(column_name='teamAbbr', group='GS')
warriors_view = CDSView(source=standings_cds, filters=[warriors_view_filter])

celtics_view_filter = GroupFilter(column_name='teamAbbr', group='BOS')
celtics_view = CDSView(source=standings_cds, filters=[celtics_view_filter])

raptors_view_filter = GroupFilter(column_name='teamAbbr', group='TOR')
raptors_view = CDSView(source=standings_cds, filters=[raptors_view_filter])

In [ ]:
west_fig = figure(
    title='Western Conference Top 2 Teams Wins Race, 2017-2018',
    plot_height=300,
    x_axis_type='datetime',
    x_axis_label='Date',
    y_axis_label='Wins',
    toolbar_location=None
)
west_fig.step(
    x='stDate', y='gameWon',
    source=standings_cds,
    view=rockets_view,
    color='#CE1141', 
    legend_label='Rockets')
west_fig.step(
    x='stDate', y='gameWon',
    source=standings_cds,
    view=warriors_view,
    color='#006BB6', 
    legend_label='Warriors')
west_fig.legend.location = 'top_left'

east_fig = figure(
    title='Eastern Conference Top 2 Teams Wins Race, 2017-2018',
    plot_height=300,
    x_axis_type='datetime',
    x_axis_label='Date',
    y_axis_label='Wins',
    toolbar_location=None
)
east_fig.step(
    x='stDate', y='gameWon',
    source=standings_cds,
    view=celtics_view,
    color='#007A33', 
    legend_label='Celtics')
east_fig.step(
    x='stDate', y='gameWon',
    source=standings_cds,
    view=raptors_view,
    color='#CE1141', 
    legend_label='Raptors')
east_fig.legend.location = 'top_left'

In [ ]:
# Use full-sized width for both figures
east_fig.plot_width = west_fig.plot_width = 800

In [ ]:
output_file('output-west-east-tabbed-layout.html')

east_panel = Panel(child=east_fig, title='Eastern Conference')
west_panel = Panel(child=west_fig, title='Western Conference')

tabs = Tabs(tabs=[east_panel, west_panel])

show(tabs)

### Adding Interactions: Selecting Data Points

In [ ]:
output_file('output-3pt-attempts-selection.html', title='3pt Attempts vs. Percentage')

three_takers_cds = ColumnDataSource(three_takers)

selected_tools = [
    'box_select',
    'lasso_select',
    'poly_select', 
    'tap',
    'reset'
]

fig = figure(
    plot_height=400,
    plot_width=600,
    x_axis_label='3pt Shots Attempted',
    y_axis_label='Percentage Made',
    title='3pt Attempts vs. Percentage',
    toolbar_location='below',
    tools=selected_tools,
)
fig.yaxis[0].formatter = NumeralTickFormatter(format='00.0%')
fig.square(
    x='play3PA', y='pct3PM',
    source=three_takers_cds,
    color='royalblue',  # Nothing is selected
    selection_color='deepskyblue',
    nonselection_color='lightgray',
    nonselection_alpha=0.3
)

show(fig)

### Adding Interactions: Hovering

In [ ]:
output_file('output-3pt-attempts-hovering.html', title='3pt Attempts vs. Percentage')

three_takers_cds = ColumnDataSource(three_takers)

selected_tools = [
    'box_select',
    'lasso_select',
    'poly_select', 
    'tap',
    'reset'
]
hover_tooltip = [ 
    ('Player', '@name'),  
    ('Three-Pointers Made', '@play3PM'),
    ('Three-Pointers Attempted', '@play3PA'),
    ('Three-Pointers Percentage', '@pct3PM{00.0%}'),    
]

fig = figure(
    plot_height=400,
    plot_width=600,
    x_axis_label='3pt Shots Attempted',
    y_axis_label='Percentage Made',
    title='3pt Attempts vs. Percentage',
    toolbar_location='below',
    tools=selected_tools,
)
fig.yaxis[0].formatter = NumeralTickFormatter(format='00.0%')
fig.square(
    x='play3PA', y='pct3PM',
    source=three_takers_cds,
    color='royalblue',  # Nothing is selected
    selection_color='deepskyblue',
    nonselection_color='lightgray',
    nonselection_alpha=0.3
)

hover_glyph = fig.circle(
    x='play3PA', y='pct3PM', source=three_takers_cds, 
    size=15,
    alpha=0, 
    hover_fill_color='black', hover_alpha=0.5
)

fig.add_tools(
    HoverTool(
        tooltips=hover_tooltip,
        renderers=[hover_glyph]
    )
)

show(fig)

### Adding Interactions: Linking Axes across multiple visualizations

In [ ]:
output_file('output-phi76ers-linked-stats.html', title='PHI 76ers Game Log')

phi76ers_gm_stats_cds = ColumnDataSource(phi76ers_gm_stats)

win_loss_mapper = CategoricalColorMapper(
    factors=['W', 'L'],  # Possible values from the winLoss column
    palette=['green', 'red'],
)

stat_names = {
    'Points': 'teamPTS',
    'Assists': 'teamAST',
    'Rebounds': 'teamTRB',
    'Turnovers': 'teamTO'
}
stat_figs = {}
for stat_label, stat_col in stat_names.items():
    fig = figure(
        plot_height=200,
        plot_width=400,
        x_range=(1, 10),  # Initially displayed game_num
        y_axis_label=stat_label,
        tools=['xpan','reset','save'],
    )
    fig.vbar(
        x='game_num', 
        top=stat_col, 
        source=phi76ers_gm_stats_cds, 
        width=0.9,
        color=dict(field='winLoss', transform=win_loss_mapper),
    )
    stat_figs[stat_label] = fig
    
phi76ers_gm_stats_grid = gridplot(
    [
        [stat_figs['Points'], stat_figs['Assists']],
        [stat_figs['Rebounds'], stat_figs['Turnovers']]
    ]
)

# Link together X-axis of each fig
stat_figs['Points'].x_range = \
    stat_figs['Assists'].x_range = \
        stat_figs['Rebounds'].x_range = \
            stat_figs['Turnovers'].x_range

show(phi76ers_gm_stats_grid)

### Adding Interactions: Linking selections

In [ ]:
output_file('output-phi76ers-linked-selections.html', title='PHI 76ers Percentages vs. Win-Loss')

phi76ers_gm_stats_points_cds = ColumnDataSource(phi76ers_gm_stats_points)

win_loss_mapper = CategoricalColorMapper(
    factors=['W', 'L'],  # Possible values from the winLoss column
    palette=['green', 'red'],
)

selected_tools = [
    'lasso_select',
    'tap',
    'reset',
    'save',
]

pct_fig = figure(
    title='2PT FG% vs 3PT FG%',
    plot_height=400,
    plot_width=400,
    tools=selected_tools,
    x_axis_label='2PT FG%',
    y_axis_label='3PT FG%', 
)
pct_fig.xaxis[0].formatter = NumeralTickFormatter(format='00.0%')
pct_fig.yaxis[0].formatter = NumeralTickFormatter(format='00.0%')
pct_fig.circle(
    x='team2P%',
    y='team3P%',
    source=phi76ers_gm_stats_points_cds,
    size=12,
    color='black'
)

tot_fig = figure(
    title='Team Points vs Opponent Points',
    plot_height=400,
    plot_width=400,
    tools=selected_tools,
    x_axis_label='Team Points',
    y_axis_label='Opponent Points', 
)
tot_fig.square(
    x='teamPTS',
    y='opptPTS',
    source=phi76ers_gm_stats_points_cds,
    size=10,
    color=dict(field='winLoss', transform=win_loss_mapper)
)

phi76ers_gm_stats_points_grid = gridplot(
    [
        [pct_fig, tot_fig],
    ]
)

show(phi76ers_gm_stats_points_grid)

### Adding Interactions: Show/Hide Data using Legends

In [ ]:
output_file('output-player-vs-player.html', title='Lebron James vs Kevin Durant')

player_stats_cds = ColumnDataSource(player_stats)

lebron_james_view_filters = [
    GroupFilter(column_name='playFNm', group='LeBron'),
    GroupFilter(column_name='playLNm', group='James'),    
]
lebron_james_view = CDSView(source=player_stats_cds, filters=lebron_james_view_filters)

kevin_durant_view_filters = [
    GroupFilter(column_name='playFNm', group='Kevin'),
    GroupFilter(column_name='playLNm', group='Durant'),    
]
kevin_durant_view = CDSView(source=player_stats_cds, filters=kevin_durant_view_filters)

common_figure_kwargs = {
    'plot_width': 400,
    'x_axis_label': 'Points',
    'toolbar_location': None,
}
common_circle_kwargs = {
    'size': 12,
    'alpha': 0.7,
    'x': 'playPTS',
    'y': 'playTRB',
    'source': player_stats_cds,    
}
lebron_james_kwargs = {
    'view': lebron_james_view,
    'color': '#002859',
    'legend_label': 'LeBron James'
}
kevin_durant_kwargs = {
    'view': kevin_durant_view,
    'color': '#FFC324',
    'legend_label': 'Kevin Durant'
}

hide_fig = figure(
    **common_figure_kwargs,
    title='Click on LEGEND to HIDE data',
    y_axis_label='Rebounds',
)
hide_fig.circle(
    **common_circle_kwargs,
    **lebron_james_kwargs
)
hide_fig.circle(
    **common_circle_kwargs,
    **kevin_durant_kwargs
)

mute_fig = figure(
    **common_figure_kwargs,
    title='Click on LEGEND to MUTE data',
    y_axis_label='Rebounds',
)
mute_fig.circle(
    **common_circle_kwargs,
    **lebron_james_kwargs,
    muted_alpha=0.1,
)
mute_fig.circle(
    **common_circle_kwargs,
    **kevin_durant_kwargs,
    muted_alpha=0.1,   
)

hide_fig.legend.click_policy = 'hide'
mute_fig.legend.click_policy = 'mute'

show(row(hide_fig, mute_fig))